In [77]:
## starting of by translating my other statcast methods
# will need to convert to the def main function

import pandas as pd
import pygsheets
from pybaseball import statcast
from pybaseball import schedule_and_record,playerid_reverse_lookup
from datetime import datetime
import time


pth="/Users/irarickman/Google Drive/Data Science/Projects/MLB Projections/Batting Average"
td=format(datetime.today(),"%Y-%m-%d")
old_data=pd.read_pickle(pth + "/lastabs.pkl")
old_data.game_date=pd.to_datetime(old_data.game_date,infer_datetime_format=True)
prev_date=old_data.game_date.max()
od=format(prev_date,"%Y-%m-%d")
if od!=td:
    new_d=statcast(od,td)
    new_data=new_d[new_d.events.notnull()]
    players_ids=playerid_reverse_lookup(new_data.batter.unique())
    id_df=players_ids[['name_last','name_first','key_mlbam']]
    new_names=new_data.merge(id_df, how = 'left',left_on='batter',right_on='key_mlbam')
    df=pd.concat([old_data,new_names])
else: 
    df=old_data
df.drop_duplicates(inplace=True)
df.to_pickle(pth + "/lastabs.pkl")
df['hit']=df.events.apply(lambda x: 1 if x in ["single",'double','home_run','triple'] else 0)
df['ab']=df.events.apply(lambda x: 0 if x in ['walk','hit_by_pitch',"caught_stealing_2b","pickoff_caught_stealing_2b",'pickoff_1b','catcher_interf','pickoff_caught_stealing_3b','pickoff_2b','pickoff_caught_stealing_home','caught_stealing_3b','caught_stealing_home',"sac_fly",'sac_bunt','sac_fly_double_play','sac_bunt_double_play'] else 1)
df['player_team']=df.apply(lambda x: x.home_team if x.inning_topbot=="Bot" else x.away_team,axis=1)
df['Opp']=df.apply(lambda x: x.away_team if x.inning_topbot=="Bot" else x.home_team,axis=1)
df['Place']=df.apply(lambda x: "Home" if x.inning_topbot=="Bot" else "Away",axis=1)
teams=df.player_team.unique()
fixers={"WSH":"WSN","CWS":"CHW"}
teams_fixed=[x if x not in fixers.keys() else fixers[x] for x in teams]

team_schedule={}
missed=[]
for t in teams_fixed:
    try:
        d=schedule_and_record(2018,t)
        d['fix_date']=d.Date.str.replace("\(\d\)","").str.strip() + " 2018"
        d['game_date']=pd.to_datetime(d.fix_date.apply(lambda x: datetime.strptime(x,"%A, %b %d %Y")).apply(lambda x: x.strftime("%Y-%m-%d")),infer_datetime_format=True)
        d['Place']=d.Home_Away.apply(lambda x: "Home" if x=="Home" else "Away")
        d2=d[d.game_date>=datetime.today()][['Place',"Opp","game_date"]]
        team_schedule[t]=d2
    except ValueError:
        print(t)
        missed.append(t)

df['name_last']=df['name_last'].str.capitalize()
df['name_first']=df['name_first'].str.capitalize()
df['player_name']=df.name_first + " " + df.name_last
sm_df=df[['game_date','game_pk','hit','ab','Opp','Place','player_name','player_team','key_mlbam']]
sm_df.sort_values(['player_name','game_date','game_pk'],inplace=True)
trim_df=sm_df.groupby(['player_name','game_date','game_pk','Opp','Place','player_team','key_mlbam']).sum().reset_index()

def player_df(player, d=trim_df):
    temp = d[d.player_name==player]
    temp=temp.sort_values(['game_date']).reset_index(drop=True)
    tm=temp.loc[len(temp)-1,'player_team']
    if tm in fixers.keys():
        sched=team_schedule[fixers[tm]]
    else:
        sched=team_schedule[tm]
    tdf= pd.concat([temp,sched])
    tdf.ab.fillna(0,inplace=True)
    tdf.hit.fillna(0,inplace=True)
    tdf.player_name.fillna(player,inplace=True)
    tdf.player_team.fillna(tm,inplace=True)
    return tdf

master_df=player_df(trim_df.player_name.unique()[0])
for p in trim_df.player_name.unique()[1:]:
    got=player_df(p)
    master_df=pd.concat([master_df,got])
master_df.game_date=master_df.game_date.apply(lambda x: format(x,"%Y-%m-%d"))

## now write to the google sheet

# #authorization
gc = pygsheets.authorize(outh_file='/Users/irarickman/client_secret.json')
mlb = 'MLB At Bats'
sh = gc.open(mlb)

#select the first sheet
wks = sh[0]

wks.set_dataframe(master_df,(1,1))

Gathering player lookup table. This may take a moment.


/Users/irarickman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [73]:
master_df.head()

,Opp,Place,ab,game_date,game_pk,hit,key_mlbam,place,player_name,player_team
0,ATL,away,2.0,2018-04-03,529483.0,1.0,595918.0,NaN,A. j. Cole,WSH
1,ATL,home,1.0,2018-04-11,529575.0,0.0,595918.0,NaN,A. j. Cole,WSH
97,ATL,NaN,0.0,2018-07-20,NaN,0.0,NaN,Home,A. j. Cole,WSH
98,ATL,NaN,0.0,2018-07-21,NaN,0.0,NaN,Home,A. j. Cole,WSH
99,ATL,NaN,0.0,2018-07-22,NaN,0.0,NaN,Home,A. j. Cole,WSH


In [69]:
master_df[master_df.player_name=="Bryce Harper"]

,Opp,Place,Tm,ab,game_date,game_pk,hit,key_mlbam,place,player_name,player_team
0,CIN,away,NaN,3.0,2018-03-30,529420.0,2.0,547180.0,NaN,Bryce Harper,WSH
1,CIN,away,NaN,3.0,2018-03-31,529444.0,0.0,547180.0,NaN,Bryce Harper,WSH
2,CIN,away,NaN,4.0,2018-04-01,529456.0,2.0,547180.0,NaN,Bryce Harper,WSH
3,ATL,away,NaN,2.0,2018-04-02,529470.0,1.0,547180.0,NaN,Bryce Harper,WSH
4,ATL,away,NaN,3.0,2018-04-03,529483.0,1.0,547180.0,NaN,Bryce Harper,WSH
5,ATL,away,NaN,3.0,2018-04-04,529496.0,0.0,547180.0,NaN,Bryce Harper,WSH
6,NYM,home,NaN,3.0,2018-04-05,529504.0,0.0,547180.0,NaN,Bryce Harper,WSH
7,NYM,home,NaN,3.0,2018-04-07,529526.0,1.0,547180.0,NaN,Bryce Harper,WSH
8,NYM,home,NaN,4.0,2018-04-08,529541.0,3.0,547180.0,NaN,Bryce Harper,WSH
9,ATL,home,NaN,1.0,2018-04-09,529548.0,0.0,547180.0,NaN,Bryce Harper,WSH


In [14]:
old_data.events.value_counts()

field_out                       44069
strikeout                       24579
single                          15699
double                           4991
home_run                         3306
force_out                        2247
grounded_into_double_play        2114
walk                             1780
sac_fly                           712
sac_bunt                          488
triple                            485
double_play                       265
hit_by_pitch                      241
fielders_choice_out               190
field_error                       187
strikeout_double_play             106
fielders_choice                    61
caught_stealing_2b                 28
pickoff_caught_stealing_2b          6
sac_fly_double_play                 6
pickoff_1b                          4
other_out                           4
catcher_interf                      3
pickoff_caught_stealing_3b          2
pickoff_2b                          2
pickoff_caught_stealing_home        1
triple_play 

In [26]:
## fixing it so its all cumulative now
## game_pk is unique game id
df=old_data
df['hit']=df.events.apply(lambda x: 1 if x in ["single",'double','home_run','triple'] else 0)
df['ab']=df.events.apply(lambda x: 0 if x in ['walk','hit_by_pitch',"caught_stealing_2b","pickoff_caught_stealing_2b",'pickoff_1b','catcher_interf','pickoff_caught_stealing_3b','pickoff_2b','pickoff_caught_stealing_home','caught_stealing_3b','caught_stealing_home',"sac_fly",'sac_bunt','sac_fly_double_play','sac_bunt_double_play'] else 1)
df['player_team']=df.apply(lambda x: x.home_team if x.inning_topbot=="Bot" else x.away_team,axis=1)
df['Opp']=df.apply(lambda x: x.away_team if x.inning_topbot=="Bot" else x.home_team,axis=1)
df['Place']=df.apply(lambda x: "home" if x.inning_topbot=="Bot" else "away",axis=1)

In [44]:
df['name_last']=df['name_last'].str.capitalize()
df['name_first']=df['name_first'].str.capitalize()
df['player_name']=df.name_first + " " + df.name_last
trim_df=df[['game_date','game_pk','hit','ab','Opp','Place','player_name','player_team','key_mlbam']]
trim_df.sort_values(['player_name','game_date','game_pk'],inplace=True)
test_df=trim_df.groupby(['player_name','game_date','game_pk','Opp','Place','player_team']).sum().reset_index()

/Users/irarickman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [47]:
test_df.head()

,player_name,game_date,game_pk,Opp,Place,player_team,hit,ab,key_mlbam
0,A. j. Cole,2018-04-03,529483.0,ATL,away,WSH,1,2,0.0
1,A. j. Cole,2018-04-11,529575.0,ATL,home,WSH,0,1,0.0
2,A. j. Ellis,2018-04-02,529462.0,COL,home,SD,0,3,0.0
3,A. j. Ellis,2018-04-03,529476.0,COL,home,SD,1,3,0.0
4,A. j. Ellis,2018-04-07,529528.0,HOU,away,SD,1,3,0.0


In [56]:
master_df=player_df(test_df.player_name.unique()[0],d=test_df)
for p in test_df.player_name.unique()[1:5]:
    got=player_df(p,test_df)
    master_df=pd.concat([master_df,got])

In [58]:
master_df.drop

,Opp,Place,Tm,ab,game_date,game_pk,hit,key_mlbam,place,player_name,player_team
0,ATL,away,NaN,2.0,2018-04-03,529483.0,1.0,0.0,NaN,A. j. Cole,WSH
1,ATL,home,NaN,1.0,2018-04-11,529575.0,0.0,0.0,NaN,A. j. Cole,WSH
97,ATL,NaN,WSN,0.0,2018-07-20,NaN,0.0,NaN,Home,A. j. Cole,WSH
98,ATL,NaN,WSN,0.0,2018-07-21,NaN,0.0,NaN,Home,A. j. Cole,WSH
99,ATL,NaN,WSN,0.0,2018-07-22,NaN,0.0,NaN,Home,A. j. Cole,WSH
100,MIL,NaN,WSN,0.0,2018-07-23,NaN,0.0,NaN,Away,A. j. Cole,WSH
101,MIL,NaN,WSN,0.0,2018-07-24,NaN,0.0,NaN,Away,A. j. Cole,WSH
102,MIL,NaN,WSN,0.0,2018-07-25,NaN,0.0,NaN,Away,A. j. Cole,WSH
103,MIA,NaN,WSN,0.0,2018-07-26,NaN,0.0,NaN,Away,A. j. Cole,WSH
104,MIA,NaN,WSN,0.0,2018-07-27,NaN,0.0,NaN,Away,A. j. Cole,WSH


In [11]:
pd.options.display.max_rows=500
pd.options.display.max_columns=100

master_df[master_df.player_name=="Juan Soto"]

,Opp,Tm,ab,game_date,hit,key_mlbam,place,player_name,player_team
0,NaN,NaN,1.0,2018-05-15,1.0,NaN,NaN,Juan Soto,WSH
1,NaN,NaN,1.0,2018-05-15,0.0,NaN,NaN,Juan Soto,WSH
2,NaN,NaN,1.0,2018-05-20,0.0,NaN,NaN,Juan Soto,WSH
3,NaN,NaN,1.0,2018-05-21,1.0,NaN,NaN,Juan Soto,WSH
4,NaN,NaN,1.0,2018-05-21,0.0,NaN,NaN,Juan Soto,WSH
5,NaN,NaN,1.0,2018-05-21,1.0,NaN,NaN,Juan Soto,WSH
6,NaN,NaN,1.0,2018-05-21,0.0,NaN,NaN,Juan Soto,WSH
7,NaN,NaN,1.0,2018-05-22,1.0,NaN,NaN,Juan Soto,WSH
8,NaN,NaN,1.0,2018-05-23,0.0,NaN,NaN,Juan Soto,WSH
9,NaN,NaN,1.0,2018-05-23,0.0,NaN,NaN,Juan Soto,WSH


In [50]:
t=pd.read_pickle('lastabs.pkl')

In [52]:
t.head()

,at_bat_number,away_score,away_team,ax,ay,az,babip_value,balls,bat_score,batter,...,type,umpire,vx0,vy0,vz0,woba_denom,woba_value,zone,name_last,name_first
0,99.0,11.0,MIL,-6.0390,20.6136,-31.5911,0.0,2.0,10.0,471865,...,S,NaN,-8.9151,-117.1824,0.5617,1.0,0.0,13.0,gonzalez,carlos
1,98.0,11.0,MIL,13.3772,25.5511,-16.0850,0.0,1.0,10.0,571448,...,S,NaN,-9.8193,-138.7981,-2.2293,1.0,0.0,3.0,arenado,nolan
2,97.0,11.0,MIL,-3.7102,17.7961,-30.0518,0.0,2.0,10.0,453568,...,X,NaN,-6.5737,-118.2398,-0.8870,1.0,0.0,9.0,blackmon,charlie
3,96.0,11.0,MIL,11.9449,24.5820,-19.3697,0.0,2.0,11.0,542583,...,X,NaN,-9.0311,-137.7384,-7.9996,1.0,0.0,7.0,aguilar,jesus
4,95.0,10.0,MIL,-1.0862,17.5596,-37.1275,1.0,2.0,10.0,543768,...,X,NaN,-6.1088,-111.0472,-0.6663,1.0,0.9,13.0,shaw,travis


In [48]:
t[t.player_name=="Bryce Harper"][['player_name','game_date']].sort_values(['player_name','game_date']).game_date.value_counts()

2018-05-19    7
2018-06-02    6
2018-05-27    5
2018-04-11    5
2018-06-24    5
2018-05-29    5
2018-03-31    5
2018-05-04    5
2018-05-02    5
2018-06-09    5
2018-05-25    5
2018-05-13    5
2018-06-17    5
2018-06-08    5
2018-05-10    5
2018-04-16    4
2018-06-10    4
2018-05-06    4
2018-05-26    4
2018-04-20    4
2018-04-08    4
2018-05-08    4
2018-05-21    4
2018-05-11    4
2018-05-03    4
2018-06-20    4
2018-06-01    4
2018-05-07    4
2018-04-28    4
2018-06-19    4
2018-04-01    4
2018-04-10    4
2018-04-13    4
2018-04-12    4
2018-05-30    4
2018-06-18    4
2018-06-05    4
2018-05-05    4
2018-06-16    4
2018-05-23    4
2018-04-22    4
2018-06-13    4
2018-04-29    3
2018-05-20    3
2018-05-22    3
2018-04-03    3
2018-04-27    3
2018-04-25    3
2018-05-28    3
2018-03-30    3
2018-06-22    3
2018-04-04    3
2018-04-07    3
2018-05-31    3
2018-05-01    3
2018-04-17    3
2018-04-24    3
2018-04-05    3
2018-05-12    3
2018-04-14    3
2018-04-18    3
2018-06-23    3
2018-06-

In [63]:
players_ids.head()

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,adams,matt,571431,adamm002,adamsma01,9393,2012.0,2018.0
1,adrianza,ehire,501303,adrie001,adriaeh01,8418,2013.0,2018.0
2,aguilar,jesus,542583,aguij001,aguilje01,11342,2014.0,2018.0
3,ahmed,nick,605113,ahmen001,ahmedni01,12147,2014.0,2018.0
4,alfaro,jorge,595751,alfaj002,alfarjo01,12180,2016.0,2018.0
